# LUME-services

### Configuration:

Set the following environment variables:

```
export LUME_MODEL_DB__HOST=127.0.0.1
export LUME_MODEL_DB__USER=root
export LUME_MODEL_DB__PASSWORD=test
export LUME_MODEL_DB__PORT=3306
export LUME_MODEL_DB__DATABASE=model_db
export LUME_MODEL_DB__CONNECTION__POOL_SIZE=1
export LUME_PREFECT__SERVER__TAG=core-1.2.4
export LUME_PREFECT__SERVER__HOST_PORT=4200
export LUME_PREFECT__SERVER__HOST=http://localhost
export LUME_PREFECT__AGENT__HOST_PORT=5000
export LUME_PREFECT__AGENT__HOST=http://localhost
export LUME_RESULTS_DB__HOST=127.0.0.1
export LUME_RESULTS_DB__PORT=27017
export LUME_RESULTS_DB__USERNAME=root
export LUME_RESULTS_DB__PASSWORD=password
export LUME_RESULTS_DB__DATABASE=test
export LUME_PREFECT__BACKEND=server 
export LUME_BACKEND=docker
export LUME_ENVIRONMENT__LOCAL_PIP_REPOSITORY=...
export LUME_ENVIRONMENT__LOCAL_CONDA_CHANNEL_DIRECTORY=...
```

Start the docker services using:
```
lume-services docker start-services
```

In [1]:
from lume_services.services.models.db import ModelDBConfig
from lume_services.services.results.mongodb import MongodbResultsDBConfig
from lume_services.services.scheduling.backends.server import (
    PrefectAgentConfig,
    PrefectConfig,
    PrefectServerConfig,
)

from lume_services.services.files.filesystems import (
    LocalFilesystem,
    MountedFilesystem,
)
from lume_services.config import LUMEServicesSettings
import logging
logging.basicConfig(level="DEBUG")

model_db_config = ModelDBConfig(
        host="127.0.0.1",
        port="3306",
        user="root",
        password="test",
        database="model_db",
)

results_db_config = MongodbResultsDBConfig(
    port="27017",
    host="127.0.0.1",
    username="root",
    database="test",
    password="password",
)

prefect_config = PrefectConfig(
    server=PrefectServerConfig(
        host="http://localhost", host_port="4200", tag="core-1.2.4"
    ),
    agent=PrefectAgentConfig(host="http://localhost", host_port="5000"),
    backend="server",
)


settings = LUMEServicesSettings(
    model_db=model_db_config,
    results_db=results_db_config,
    prefect=prefect_config,
    backend="docker",
    mounted_filesystem=None,
)

## Use LUME-services config to create injected services

In [2]:
from lume_services import config
config.configure(settings)

# initialize singletons
model_db_service = config.context.model_db_service()
results_db_service = config.context.results_db_service()
scheduling_service = config.context.scheduling_service()

INFO:lume_services.config:Configuring LUME-services environment...
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
INFO:lume_services.config:Environment configured.
DEBUG:lume_services.config:Environment configured using {'model_db': {'host': '127.0.0.1', 'port': 3306, 'user': 'root', 'database': 'model_db', 'connection': {'pool_size': None, 'pool_pre_ping': True}, 'dialect_str': 'mysql+pymysql'}, 'results_db': {'username': 'root', 'host': '127.0.0.1', 'port': 27017, 'authMechanism': 'DEFAULT'}, 'prefect': {'server': {'tag': 'core-1.2.4', 'host': 'http://localhost', 'host_port': '4200', 'host_ip': '127.0.0.1'}, 'ui': {'host': 'http://localhost', 'host_port': '8080', 'host_ip': '127.0.0.1', 'apollo_url': 'http://localhost:4200/graphql'}, 'telemetry': {'enabled': True}, 'agent': {'host': 'http://localhost', 'host_port': '5000'}, 'home_dir': '~/.p

## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [ ]:
#model_db_service._reset()

## Store example model, deployment, flow, and project

In [3]:
from lume_services.models import Model

model = Model.create_model(
    author = "Jackie Garrahan",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu_hxr",
    description = "test_model"
)

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.


In [4]:
# create a project
project_name = model_db_service.store_project(
    project_name="test", description="my_description"
)

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO project (project_name, description) VALUES (:project_name, :description)
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO project (project_name, description) VALUES (:project_name, :description)


In [5]:
scheduling_service.create_project("test")

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 68
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 74


## Let's use the source of the model we created in the demo

In [6]:
source_path = "https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.9/my_model-0.0.9.tar.gz"
# populates local channel
model.store_deployment(source_path, project_name="test")

INFO:lume_services.environment.solver:https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.9/my_model-0.0.9.tar.gz saved to /tmp/lume-services/my_model-0.0.9.tar.gz
INFO:lume_services.environment.solver:https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.9/my_model-0.0.9.tar.gz saved to /tmp/lume-services/my_model-0.0.9.tar.gz
INFO:lume_services.environment.solver:Using conda channels conda-forge to install dependencies.
DEBUG:conda.cli.python_api:executing command >>>  conda create -p /tmp/lume-services/tmp_env
DEBUG:conda.common.io:overtaking stderr and stdout
DEBUG:conda.core.solve:solving prefix /tmp/lume-services/tmp_env
  specs_to_remove: frozenset()
  specs_to_add: frozenset()
  prune: Null
DEBUG:conda.resolve:restricting to unmanageable packages: __osx
DEBUG:conda.resolve:restricting to unmanageable packages: __unix
DEBUG:conda.resolve:restricting to unmanageable packages: __archspec
DEBUG:conda.resolve:Retrieving packages for: 
DEBUG:conda.co

DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/platformdirs-2.5.2-pyhd8ed1ab_1.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/importlib-metadata-4.11.4-py39h6e9494a_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/toml-0.10.2-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/mistune-0.8.4-py39h89e85a6_1005.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/typepy-1.3.0-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/freetype-2.12.1-h3f81eb7_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Use

DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/mergedeep-1.3.4-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/glob2-0.7-py_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/python-dateutil-2.8.2-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/mypy_extensions-0.4.3-py39h6e9494a_5.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/libopenblas-0.3.21-openmp_h429af6e_3.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/cached_property-1.5.2-pyha770c72_1.json
DEBUG:conda.core.prefix_data:loading prefix record

DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/requests-2.28.1-pyhd8ed1ab_1.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/webencodings-0.5.1-py_1.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/coverage-6.4.4-py39h6218fd2_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/pluggy-1.0.0-py39h6e9494a_3.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/mccabe-0.7.0-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/zlib-1.2.12-hfe4f2af_2.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/m

DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/ruamel_yaml-0.15.80-py39h701faf5_1007.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/flake8-5.0.4-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/unicodedata2-14.0.0-py39h63b48b0_1.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/libsqlite-3.39.3-ha978bb4_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/executing-1.0.0-pyhd8ed1ab_0.json
DEBUG:conda.core.prefix_data:loading prefix record /Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/conda-meta/patch-2.7.6-hbcf498f_1002.json
DEBUG:conda.core.prefix_data:loading prefix record /User

...

  Running command python setup.py egg_info
  running egg_info
  creating /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-l85qzpim/my_model.egg-info
  writing /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-l85qzpim/my_model.egg-info/PKG-INFO
  writing dependency_links to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-l85qzpim/my_model.egg-info/dependency_links.txt
  writing entry points to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-l85qzpim/my_model.egg-info/entry_points.txt
  writing requirements to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-l85qzpim/my_model.egg-info/requires.txt
  writing top-level names to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-l85qzpim/my_model.egg-info/top_level.txt
  writing manifest file '/private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-l85qzpim/my_model.egg-info/

INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO deployment (version, package_import_name, asset_dir, source, sha256, image, is_live, model_id) VALUES (:version, :package_import_name, :asset_dir, :source, :sha256, :image, :is_live, :model_id)
INFO:lume_services.services.models.db.db:ModelDB inserting many: ['INSERT INTO deployment_dependencies (name, source, local_source, version, deployment_id, dependency_type_id) VALUES (:name, :source, :local_source, :version, :deployment_id, (SELECT dependency_type.id \nFROM dependency_type \nWHERE dependency_type.type = :type_1))', 'INSERT INTO deployment_dependencies (name, source, local_source, version, deployment_id, dependency_type_id) VALUES (:name, :source, :local_source, :version, :deployment_id, (SELECT dependency_type.id \nFROM dependency_type \nWHERE dependency_type.type = :type_1))', 'INSERT INTO deployment_dependencies (name, source, local_source, version, deployment_id, dependency_type_id) VALUES (:name, :sou

DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.
INFO:lume_services.services.models.db.db:Sucessfully executed: ['INSERT INTO deployment_dependencies (name, source, local_source, version, deployment_id, dependency_type_id) VALUES (:name, :source, :local_source, :version, :deployment_id, (SELECT dependency_type.id \nFROM dependency_type \nWHERE dependency_type.type = :type_1))', 'INSERT INTO deployment_dependencies (name, source, local_source, version, deployment_id, dependency_type_id) VALUES (:name, :source, :local_source, :version, :deployment_id, (SELECT dependency_type.id \nFROM dependency_type \nWHERE dependency_type.type = :type_1))', 'INSERT INTO deployment_dependencies (name, source, local_source, version, deployment_id, dependency_type_id) VALUES (:name, :source, :local_source, :version, :deployment_id, (SELECT dependency_type.id \nFROM dependency_type \nWHERE dependency_type.type = :type_1))

INFO:lume_services.services.scheduling.backends.server:Flow run config is not empty. Clearing existing labels and assigning                     new.
/Users/jacquelinegarrahan/miniconda3/envs/lume-services-dev/lib/python3.9/site-packages/prefect/core/flow.py:1726: UserWarning: No result handler was specified on your Flow. Cloud features such as input caching and resuming task runs from failure may not work properly.
  registered_flow = client.register(
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 69
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 94
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBUG:urllib3.connectionpool:http://localhost:4200 "POST / HTTP/1.1" 200 45
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4200
DEBU

Flow URL: http://localhost:8080/default/flow/44fd2250-6e42-461b-b357-9956e5444bdb
 └── ID: ceccf514-e3e1-46cc-a043-49da0c072a3b
 └── Project: test
 └── Labels: ['lume-services']


In [ ]:
model.run(parameters={
                        "input1": "hey", 
                        "input2": "jackie", 
                        "filename": f"/Users/jgarra/sandbox/lume-services/test_file.txt", 
                        "filesystem_identifier":"local"}
         )

In [ ]:
model.results()

In [11]:
from lume_services.models import Model

model = Model(model_id=1)

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.


In [12]:
model.load_deployment()

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.sha256, deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.image, deployment.is_live, deployment.model_id 
FROM deployment 
WHERE deployment.model_id = :model_id_1 ORDER BY deployment.deploy_date DESC
DEBUG:lume_services.services.models.db.db:ModelDB creating session.
DEBUG:lume_services.services.models.db.db:ModelDB session created.
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment_dependencies._id, deployment_dependencies.name, deployment_dependencies.source, deployment_dependencies.local_source, deployment_dependencies.version, deployment_dependencies.deployment_id, deployment_dependencies.dependency_type_id, dependency_type_1.id, dependency_type_1.type 
FROM deployment_dependencies LEFT OUTER JOIN dependency_type AS dependency_type_1 ON dependency_type_1.id = deployment_depend

In [14]:
model.deployment.flow.prefect_flow.run(**{
                        "input1": 1, 
                        "input2": 2, 
                        "filename": "/Users/jgarra/sandbox/lume-services/test_file.txt", 
                        "filesystem_identifier":"local"}
)

[2022-09-07 15:08:58-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'my-model'


INFO:prefect.FlowRunner:Beginning Flow run for 'my-model'


[2022-09-07 15:08:58-0700] INFO - prefect.TaskRunner | Task 'input2': Starting task run...


INFO:prefect.TaskRunner:Task 'input2': Starting task run...


[2022-09-07 15:08:58-0700] INFO - prefect.TaskRunner | Task 'input2': Finished task run for task with final state: 'Success'


...

[2022-09-07 15:08:58-0700] INFO - prefect.TaskRunner | Task 'save_db_result': Finished task run for task with final state: 'TriggerFailed'


INFO:prefect.TaskRunner:Task 'save_db_result': Finished task run for task with final state: 'TriggerFailed'


[2022-09-07 15:08:58-0700] INFO - prefect.FlowRunner | Flow run FAILED: some reference tasks failed.


INFO:prefect.FlowRunner:Flow run FAILED: some reference tasks failed.


<Failed: "Some reference tasks failed.">

In [13]:
model

Model(metadata=Model(                     model_id=1,                     created=datetime.datetime(2022, 9, 7, 22, 2, 3),                     author='Jackie Garrahan'),                     laboratory='slac',                     facility='lcls',                     beampath='cu_hxr',                     description='test_model'                 ), deployment=Deployment(metadata=None, project=Project(metadata=Project(                 project_name='test',                 description='my_description',                 )), dependencies=[Dependencies(                     id=1,                     name='_libgcc_mutex',                     source='https://conda.anaconda.org/t/jr-ac5b1b42-c470-4696-a06b-9e98c0d40429/conda-forge/linux-64/_libgcc_mutex-0.1-conda_forge.tar.bz2',                     local_source='/Users/jacquelinegarrahan/sandbox/lume-services/dev/local_conda_channel/linux-64/_libgcc_mutex-0.1-conda_forge.tar.bz2'                     version='0.1'                 ), Dependencies(   